# Exercise 3: Reading and Writing NetCDF files with Python

## Aim: Introduce the netCDF4 library in Python to read and create NetCDF4 files.

### Issues covered:

- Importing netCDF4
- Reading a NetCDF file as a Dataset instance
- Accessing Dimensions, Variables, and Attributes
- Defining Dimensions, Variables, and Attributes
- Writing a NetCDF file as a Dataset

## Let's work with the netCDF4 library to examine the contents of a data file.

Import the `netCDF4` library

In [2]:
import netCDF4 as nc
nc

<module 'netCDF4' from '/opt/jaspy/lib/python3.10/site-packages/netCDF4/__init__.py'>

Open the file "../example_data/ggas2014121200_00-18.nc" with the netCDF4 `Dataset` method,
assign it to the `ds` variable.

In [ ]:
ds = nc.Dataset('../example_data/ggas2014121200_00-18.nc')
ds.variables

Loop through and print Dataset `variables` names, this is the ID that act like the key to access the Variable.

In [17]:
for key in ds.variables.keys(): print(key)

longitude
latitude
surface
time
CI
SSTK
MSL
TCC
U10
V10
SKT


In [20]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    CDI: Climate Data Interface version 1.5.6 (http://code.zmaw.de/projects/cdi)
    Conventions: CF-1.4
    history: Fri Apr 10 10:42:46 2015: cdo selname,SSTK,CI,U10,V10,TCC,SKT,MSL example_data/ggas2014121200_00-18_all_vars.nc example_data/ggas2014121200_00-18.nc
Fri Apr 10 10:42:13 2015: cdo mergetime /badc/ecmwf-era-interim/data/gg/as/2014/12/12/ggas201412120000.nc /badc/ecmwf-era-interim/data/gg/as/2014/12/12/ggas201412120600.nc /badc/ecmwf-era-interim/data/gg/as/2014/12/12/ggas201412121200.nc /badc/ecmwf-era-interim/data/gg/as/2014/12/12/ggas201412121800.nc example_data/ggas2014121200_00-18_all_vars.nc
Sun Jan 11 11:15:19 GMT 2015 - CONVSH V1.92 16-February-2006
    CDO: Climate Data Operators version 1.5.6.1 (http://code.zmaw.de/projects/cdo)
    dimensions(sizes): longitude(512), latitude(256), surface(1), time(4)
    variables(dimensions): float32 longitude(longitude), float32 lat

From the Dataset `variables`, assign the key `SSTK` to the `sst` variable.
Access the `SSTK` variable like you would a dictionary from `ds.variables`.

- Print the `shape` and `size` of the `sst` variable
- Loop through the `dimensions` of `sst` and print the dimension name and length.
- Loop through the NetCDF attributes of `sst` and print the name and value.
(use `sst.ncattrs()` to access the attributes and `getattr(sst, {attribute name})` to get the value)

In [42]:
sst = ds.variables['SSTK']
sst.shape, sst.size

((4, 1, 256, 512), 524288)

In [50]:
for dim in sst.dimensions:
    print(dim, len(dim))

time 4
surface 7
latitude 8
longitude 9


In [134]:
sst

<class 'netCDF4._netCDF4.Variable'>
float32 SSTK(time, surface, latitude, longitude)
    long_name: Sea surface temperature
    units: K
    grid_type: gaussian
    _FillValue: 2e+20
    source: GRIB data
    name: SSTK
    title: Sea surface temperature
    date: 12/12/14
    time: 00:00
unlimited dimensions: time
current shape = (4, 1, 256, 512)
filling on

In [133]:
sst = ds.variables["SSTK"]

print(sst.shape, sst.size)

for d in sst.dimensions:
    print(d, len(d))

for attr in sst.ncattrs():
    print(f"{attr} = {getattr(sst, attr)}")

(4, 1, 256, 512) 524288
time 4
surface 7
latitude 8
longitude 9
long_name = Sea surface temperature
units = K
grid_type = gaussian
_FillValue = 2.0000000400817547e+20
source = GRIB data
name = SSTK
title = Sea surface temperature
date = 12/12/14
time = 00:00


In [76]:
import numpy as np

## Let's extract some data and its related coordinate information and metadata.

Using the `sst` variable from before, take a slice of `sst` as follows and assign it the variable `arr`:

```python
sst[1, 0, 10:20, 30:35]
```

- Print what type of object `arr` is

In [82]:
import numpy as np

In [138]:
arr = sst[1,0,10:20, 30:35]
print(type(arr))
# print(f"{np.array2string(arr, formatter={'float': lambda x: f'{x:.2f}'})}")

<class 'numpy.ma.core.MaskedArray'>


Assign and print the list of `dimensions` from `sst` to the variable `dims`.
Assign the `variables` dictionary of the Dataset, `ds`, to the variables `vars`

In [97]:
dims = sst.dimensions

vars = ds.variables

print(dims)

('time', 'surface', 'latitude', 'longitude')


Now extract the slices from each Dataset variable in `vars` matching those in `arr`
(i.e. matching the values in slice `[1, 0, 10:20, 30:35]` to the values in list `dims`).

Assign them to the following variables:

- Assign `time` slice to `arr_time`
- Assign `surface` slice to `arr_level`
- Assign `latitude` slice to `arr_lats`
- Assign `longitude` slice to `arr_lons`

Print the four new variables.

In [106]:
vars['time'].shape,vars['surface'].shape,vars['latitude'].shape,vars['longitude'].shape

((4,), (1,), (256,), (512,))

In [101]:
arr_time = vars['time'][1]
arr_level = vars['surface'][0]
arr_lats = vars['latitude'][10:20]
arr_lons = vars['longitude'][30:35]

In [116]:
vars['surface'][:]

masked_array(data=[0.],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [121]:
print('at time  ',arr_time, '\n',
      'and depth = ',arr_level,'\n',
      'the longitude are ', arr_lats, '\n',
      'and the latitudes are', arr_lons, '\n')

at time   0.25 
 and depth =  0.0 
 the longitude are  [82.45532  81.75363  81.05194  80.350235 79.64853  78.94681  78.245094
 77.543365 76.84164  76.13991 ] 
 and the latitudes are [21.09375  21.796875 22.5      23.203125 23.90625 ] 



Create an empty dictionary called `metadata`, Loop through the NetCDF Variable attributes of `sst` and copy them into this new dictionary.
Use the method as before to get name and value from `sst` and assign them to the key and value of the dictionary.

Print the dictionary.

In [132]:
sst.ncattrs()

['long_name',
 'units',
 'grid_type',
 '_FillValue',
 'source',
 'name',
 'title',
 'date',
 'time']

In [147]:
ds.variable['SSTK']


AttributeError: NetCDF: Attribute not found

In [151]:
!ncdump -h ../example_data/ggas2014121200_00-18.nc

netcdf ggas2014121200_00-18 {
dimensions:
	longitude = 512 ;
	latitude = 256 ;
	surface = 1 ;
	time = UNLIMITED ; // (4 currently)
variables:
	float longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "longitude" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	float surface(surface) ;
		surface:long_name = "surface" ;
		surface:units = "level" ;
		surface:axis = "Z" ;
	double time(time) ;
		time:standard_name = "time" ;
		time:units = "days since 2014-12-12 00:00:00" ;
		time:calendar = "standard" ;
	float CI(time, surface, latitude, longitude) ;
		CI:long_name = "Sea-ice cover" ;
		CI:units = "(0 - 1)" ;
		CI:grid_type = "gaussian" ;
		CI:_FillValue = 2.e+20f ;
		CI:source = "GRIB data" ;
		CI:name = "CI" ;
		CI:title = "Sea-ice cover" ;
		CI:date = "12/12/14" ;
		CI:ti

## Let's write the data/metadata from the previous section to a new NetCDF file

In this section, we will define our own Variables, Dimensions, Attributes and save them as a NetCDF file.

To start, import numpy as np.

In [173]:
from netCDF4 import Dataset
import numpy as np
from numpy.random import uniform
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num

Open a new netCDF `Dataset` for writing to a file: "mydata.nc", specify the `mode` as write with "w" and the `format` as "NETCDF4_CLASSIC"
Assign this to the variable `myds`

In [174]:
pwd()

'/home/users/train023/my-isc-work/python-data/notebooks'

In [175]:
myds = Dataset('mydata.nc', 'w', format='NETCDF4_CLASSIC')

Create four new Dimensions to `myds` from your previous slices. Re-use the names from the last section.
Note that the "level" and "time" Dimensions should have length, "lat" length 10 and "lon" length 5.
To create a new Dimension use `myds.createDimension(name, size)`

In [176]:
time = myds.createDimension('time', 1)
level = myds.createDimension('level', 1)
lat = myds.createDimension('lat', 10)
lon = myds.createDimension('lon', 5)

Create four Variables from those dimensions and assign them following this example for times:

```python
times = myds.createVariable('times', np.float64, ('time',))
```

In [178]:
times = myds.createVariable('times', np.float64, ('time',))
levels = myds.createVariable('levels', np.float64, ('level',))
lats = myds.createVariable('lats',  np.float64, ('lat',))
lons = myds.createVariable('lons',  np.float64, ('lon',))

RuntimeError: NetCDF: String match to name in use

Create `myvar` as a new Dataset Variable, with id "temp", type "np.float64", and dimensions: "time", "level", "lat", "lon".

In [179]:
myvar = myds.createVariable("temp", np.float64, ('time', 'level', 'lat', 'lon',))

Remove the `_FillValue` value in the `metadata` dictionary.
The next step will not work unless we do this.
Fill values should be handled when the Variable is created, but we are ignoring that for this example.

Use `del` to remove the `_FillValue` from `metadata`

In [184]:
del metadata['_FillValue']

NameError: name 'metadata' is not defined

Write Variable attributes from the key/value pairs found in the input data (held in the `metadata` dictionary) to `myvar`.
Use `myvar.setncattr(key, value)` to write attributes to the Dataset.

Write one more global attribute "source" to `myds`. Set the attribute `source` to the value "super dataset".
Use `myds.source` to create and set the value to the global attribute.

In [185]:
for (key, value) in metadata.items():
    myvar.setncattr(key, value)

myds.source = "super dataset"


NameError: name 'metadata' is not defined

Assign the `arr` array from before to `myvar[:]`

Write some data values to each of your four spatio-temporal variables you created.
Use simple lists of integers for these.
Make sure they are the right length matching the slices from the previous section,
use the index `[:]` on your `myds` Variables and assign the created array variables to them.

Lastly close the Dataset, `myds`, to save the file.

In [168]:
myvar[:] = ...


myds.close()


NameError: name 'myvar' is not defined

To view the file contents, you can use the Jasmin service to run the `$ ncdump mydata.nc` command from the directory where the file is saved. Alternatively, you can call out to the linux shell from a Notebook, using: 

```
!ncdump mydata.nc
```